In [247]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [248]:
data = pd.read_csv('D:\Data Science\Learning/movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
758,tt1568911,66000000,177584879,War Horse,Tom Hiddleston|Benedict Cumberbatch|Toby Kebbe...,Steven Spielberg,Separated by War. Tested by Battle. Bound by F...,Follows a young man named Albert and his horse...,146,Drama|War,DreamWorks SKG|Amblin Entertainment|Reliance E...,12/25/2011,6.9,2011
168,tt2467046,16000000,19682924,Left Behind,Nicolas Cage|Chad Michael Murray|Lea Thompson|...,Vic Armstrong,The End Begins,A small group of survivors are left behind aft...,110,Thriller|Action|Science Fiction,Stoney Lake Entertainment,10/3/2014,3.8,2014
1404,tt0383222,25000000,2405420,BloodRayne,Kristanna Loken|Ben Kingsley|Michelle Rodrigue...,Uwe Boll,Revenge never tasted so sweet.,"In eighteenth century Romania, Rayne, a dhampi...",95,Action|Adventure|Fantasy|Horror,Boll KG,10/22/2005,3.8,2005
1261,tt1333125,6000000,32438988,Movie 43,Dennis Quaid|Greg Kinnear|Common|Charlie Saxto...,Steven Brill|Elizabeth Banks|Steve Carr|Rusty ...,The biggest cast ever assembled for the most o...,The film is composed of multiple comedy shorts...,90,Comedy,Greenestreet Films|Relativity Media|Virgin Pro...,1/1/2013,4.4,2013
870,tt0264464,52000000,352114312,Catch Me If You Can,Leonardo DiCaprio|Tom Hanks|Christopher Walken...,Steven Spielberg,The true story of a real fake.,Frank W. Abagnale Jr. is a cunning con man â€“...,141,Drama|Crime,DreamWorks SKG|Kemp Company|Splendid Pictures,12/25/2002,7.6,2002


In [249]:
display(data.describe())
display(data.describe(include = ['object']))
display(data.info())

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt0468569,The Gift,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Steven Soderbergh,Misery loves family.,A pampered Beverly Hills chihuahua named Chloe...,Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


None

# Предобработка

In [250]:
# создадим словарь для ответов
answers = {} 

# Создаем новый столбец, в который записываем прибыль фильма, равную разниче между сборами и бюджетом
data['profit'] = data.revenue - data.budget

# Создаем новый столбец, в который записываем месяц выхода фильма
data['release_month'] = pd.to_datetime(data.release_date).dt.month

In [251]:
# Создаем функцию для "распаковки" значений ячеек, в которых данные идут сроковым перечислением, разделенным "|"
def split_explode(data, i_col):
    
    # Присваиваем переменной значение строкового названия столбца
    col_f = data.columns[i_col]
    
    # Преобразовываем значения ячеек столбца 'col_f' в строки, далее в список, в качестве разделителя указываем '|'
    data[col_f] = data[col_f].apply(lambda x: str(x).split('|'))
    
    # "Распаковываем" списки столбца 'director' в отдельные значения с сохранением индекса строк
    data = data.explode(col_f)
    
    return data

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [252]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [253]:
# тут пишем ваш код для решения данного вопроса:
display(data[data.budget == data.budget.max()].original_title)
display(data[data.budget == data.budget.max()].imdb_id)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

723    tt1298650
Name: imdb_id, dtype: object

ВАРИАНТ 2

In [254]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [255]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [256]:
# Выводим название фильма с максимальной длительностью
data[data.runtime == data.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [257]:
# Выводим название фильма с минимальной длительностью
display(data[data.runtime == data.runtime.min()].original_title)

# Записываем в словарь ответ на третий вопрос
answers['3'] = '3. Winnie the Pooh (tt1449283)'

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [258]:
# Выводим среднее значение длительности фильмов, округленное до целого значения
display(round(data.runtime.mean(), 0))

# Записываем в словарь ответ на четвертый вопрос
answers['4'] = '2. 110'

110.0

# 5. Каково медианное значение длительности фильмов? 

In [259]:
# Выводим медианное значение длительности фильмов, округленное до целого значения
display(round(data.runtime.median(), 0))

# Записываем в словарь ответ на пятый вопрос
answers['5'] = '1. 107'

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [260]:
# Выводим название фильма с максимальным значением прибыли
display(data[data.profit == data.profit.max()].original_title)

# Записываем в словарь ответ на шестой вопрос
answers['6'] = '5. Avatar (tt0499549)'

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [261]:
# Выводим название фильма с максимальным значением прибыли
display(data[data.profit == data.profit.min()].original_title)

# Записываем в словарь ответ на седьмой вопрос
answers['7'] = '5. The Lone Ranger (tt1210819)'

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [262]:
# Выводим количество уникальных значений идентификаторов фильмов у который сборы превысили бюджет
display(data[data.revenue > data.budget].imdb_id.nunique())

# Записываем в словарь ответ на восьмой вопрос
answers['8'] = '1. 1478'

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [263]:
# Записываем в переменную датасет с фильмами, которые вышли в 2008 году
a = data[data.release_year == 2008]

# Из полученного датасета выводим название фильма с максимальными сборами 
display(a[a.revenue == a.revenue.max()].original_title)

# Из полученного датасета выводим идентификатор фильма с максимальными сборами
display(a[a.revenue == a.revenue.max()].imdb_id)

# Записываем в словарь ответ на девятый вопрос
answers['9'] = '4. The Dark Knight (tt0468569)'

599    The Dark Knight
Name: original_title, dtype: object

599    tt0468569
Name: imdb_id, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [264]:
# Записываем в переменную датасет с фильмами, которые вышли с 2012 по 2014 включительно
b = data[(data.release_year >= 2012) & (data.release_year <= 2014)]

# Из полученного датасета выводим название фильма с минимальной прибылью
display(b[b.profit == b.profit.min()].original_title)

# Из полученного датасета выводим идентификатор фильма с минимальной прибылью
display(b[b.profit == b.profit.min()].imdb_id)

# Записываем в словарь ответ на десятый вопрос
answers['10'] = '5. The Lone Ranger (tt1210819)'

1245    The Lone Ranger
Name: original_title, dtype: object

1245    tt1210819
Name: imdb_id, dtype: object

# 11. Какого жанра фильмов больше всего?

In [265]:
# Копируем данные для чистоты эксперимента
data_11 = data.copy()

# Предобрабатываем столбец 'genres' 
data_11 = split_explode(data_11, 9)

# Группируем данные по жанрам, подсчитываем количество значений, сортируем в порядке убывания
display(data_11.groupby('genres').genres.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на одиннадцатый вопрос
answers['11'] = '3. Drama'

'Drama'

ВАРИАНТ 2

In [266]:
# Копируем исходные данные для чистоты эксперимента
data_g = pd.read_csv('D:\Data Science\Learning/movie_bd_v5.csv')

'''Преобразуем все значения столбца 'genres' в единую строку(str), разделяя элементы через '|', 
    преобразовываем строку в список(list), указав в качестве разделителя '|',
    преобразовываем список в серию(Series) и считаем количество раз, которое встречается каждый жанр из списка'''
pd.Series(data_g.genres.str.cat(sep='|').split('|')).value_counts().index[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [267]:
# Копируем данные для чистоты эксперимента
data_12 = data.copy()

# Предобрабатываем столбец 'genres' 
data_12 = split_explode(data_12, 9)

# По жанрам группируем фильмы, прибыль которых больше 0, считаем их количество, значения сортируем в порядке убывания
display(data_12[data_12.profit > 0].groupby('genres').genres.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на двенадцатый вопрос
answers['12'] = '1. Drama'

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [268]:
# Копируем данные для чистоты эксперимента
data_13 = data.copy()

# Предобрабатываем столбец 'director' 
data_13 = split_explode(data_13, 5)

# Суммируем кассовыые сборы фильмов, группируем по режиссерам, сортируем в порядке убывания, выводим индекс первой строки
display(data_13.groupby('director').revenue.sum().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на тринадцатый вопрос
answers['13'] = '5. Peter Jackson'

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [269]:
# Копируем данные для чистоты эксперимента
data_14 = data.copy()

# Предобрабатываем столбец 'genres' 
data_14 = split_explode(data_14, 9)

# Предобрабатываем столбец 'director' 
data_14 = split_explode(data_14, 5)

# Считаем количество фильмов в жанре 'Action', группируем по режиссерам, сортируем по убыванию, выводим индекс первой строки
display(data_14[data_14.genres == 'Action'].groupby('director').director.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на четырнадцатый вопрос
answers['14'] = '3. Robert Rodriguez'

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [270]:
# Копируем данные для чистоты эксперимента
data_15 = data.copy()

# Предобрабатываем столбец 'cast' 
data_15 = split_explode(data_15, 4)

# Отбираем фильмы, выпущенные в 2012, группируем по актерам, суммируем кассовые сборы, сортируем по убыванию, выводим первую строку
display(data_15[data_15.release_year == 2012].groupby('cast').revenue.sum().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на четырнадцатый вопрос
answers['15'] = '3. Chris Hemsworth'

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [271]:
# Копируем данные для чистоты эксперимента
data_16 = data.copy()

# Предобрабатываем столбец 'cast' 
data_16 = split_explode(data_16, 4)

# Отбираем фильмы с бюджетом выше среднего, считаем количество фильмов в котором снялся каждый актер, сортируем по убыванию
display(data_16[data_16.budget > data_16.budget.mean()].groupby('cast').imdb_id.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на шестнадцатый вопрос
answers['16'] = '3. Matt Damon'

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [272]:
# Копируем данные для чистоты эксперимента
data_17 = data.copy()

# Предобрабатываем столбец 'genres' 
data_17 = split_explode(data_17, 9)

# Предобрабатываем столбец 'cast' 
data_17 = split_explode(data_17, 4)

# Отбираем фильмы с участием 'Nicolas Cage', группируем по жанрам, считаем количество, сортируем по убыванию, выводим индекс первой строки
display(data_17[data_17.cast == 'Nicolas Cage'].groupby('genres').imdb_id.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на семнадцатый вопрос
answers['17'] = '2. Action'

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [273]:
# Копируем исходные данные
data_18 = data.copy()

# Предобрабатываем столбец 'production_companies' 
data_18 = split_explode(data_18, 10)

# Оставляем в датасете только фильмы компании Paramount Pictures
data_18 = data_18[data_18.production_companies == 'Paramount Pictures']

# Отбираем из датасета фильм с минимальной прибылью
display(data_18[data_18.profit == data_18.profit.min()].head(1))

# Записываем в словарь ответ на восемнадцатый вопрос
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures,7/19/2002,6.0,2002,-64831034,7


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [274]:
# Копируем исходные данные
data_19 = data.copy()

# Группируем по дате выхода и суммируем кассовые сборы, сортируем в порядке убывания
display(data_19.groupby('release_year').revenue.sum().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на девятнадцатый вопрос
answers['19'] = '5. 2015'

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [275]:
# Копируем исходные данные
data_20 = data.copy()

# Предобрабатываем столбец 'production_companies' 
data_20 = split_explode(data_20, 10)

# Отбираем фильмы компаний Warner Bros, группируем по годам выпуска и суммируем прибыль
display(data_20[data_20.production_companies.str.contains('Warner Bros')].groupby('release_year').profit.sum().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на двадцатый вопрос
answers['20'] = '1. 2014'

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [276]:
# Копируем исходные данные
data_21 = data.copy()

# Считаем количество вышедших фильмов в каждом месяце, сортируем по убыванию, показываем первый индекс в полученной серии 
display(data_21.groupby('release_month').imdb_id.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на двадцать первый вопрос
answers['21'] = '4. Сентябрь'

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [277]:
# Копируем исходные данные
data_22 = data.copy()

# Отбираем фильмы, которые вышли летом, считаем их количество
display(data_22[(data_22.release_month >= 6) & (data_22.release_month <= 8)].imdb_id.count())

# Записываем в словарь ответ на двадцать второй вопрос
answers['22'] = '2. 450'

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [278]:
# Копируем исходные данные
data_23 = data.copy()

# Отбираем фильмы, которые выпущены в зимние месяцы
data_23 = data_23[(data_23.release_month == 12) | (data_23.release_month == 1) | (data_23.release_month == 2)]

# Предобрабатываем столбец 'director' 
data_23 = split_explode(data_23, 5)

# Группируем фильмы по режиссерам, считаем их количество, сортируем по убыванию, выводим индекс первой строки
display(data_23.groupby('director').imdb_id.count().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на двадцать третий вопрос
answers['23'] = '5. Peter Jackson'

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [279]:
# Копируем исходные данные
data_24 = data.copy()

# Предобрабатываем столбец 'production_companies' 
data_24 = split_explode(data_24, 10)

# Создаем новый столбец, в который сохраняем длину названия фильма
data_24['title_length'] = data_24.original_title.str.len()

# Группируем по студиям, находим среднее значение длин фильмов, сортируем в порядке убывания, выводим индекс первой строки
display(data_24.groupby('production_companies').title_length.mean().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на двадцать четвертый вопрос
answers['24'] = '5. Four By Two Productions'

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [280]:
# Копируем исходные данные
data_25 = data.copy()

# Предобрабатываем столбец 'production_companies' 
data_25 = split_explode(data_25, 10)

# Создаем новый столбец, в который сохраняем длину описания фильма
data_25['overview_length'] = data_25.overview.str.len()

# Группируем по студиям, находим среднее значение длин фильмов, сортируем в порядке убывания, выводим индекс первой строки
display(data_25.groupby('production_companies').overview_length.mean().sort_values(ascending=False).index[0])

# Записываем в словарь ответ на двадцать пятый вопрос
answers['25'] = '3. Midnight Picture Show'

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [281]:
# Копируем исходные данные
data_26 = data.copy()

# Отсекаем от датасета фильмы, рейтинг которых менее 99%, выводим столбцы 'original_title', 'vote_average' датасета 
display(data_26[data_26.vote_average > data_26.vote_average.quantile(0.99)][['original_title', 'vote_average']])

# Записываем в словарь ответ на двадцать шестой вопрос
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [282]:
# Копируем исходные данные
data_27 = data.copy()

# Преобразовываем значения ячеек столбца 'cast' в строки, далее в список, в качестве разделителя указываем '|'
data_27.cast = data_27.cast.apply(lambda x: str(x).split('|'))

# Создаем новый столбец, в котором храним списки комбинаций актеров
data_27['cast_pairs'] = data_27.cast.apply(lambda y: list(combinations(y, 2)))

# Создаем счетчик пар актеров
pairs = Counter()

# Проходим циклом по массиву
for i in range(0, len(data_27)):
    
    # Увеличиваем счетчик пар результатом обработки ячейки столбца 'cast_pairs' с индексом i
    pairs.update(Counter(data_27.cast_pairs[i]))

# Выводим 5 наиболее часто встречающихся пар актеров
display(pairs.most_common(5))

# Записываем в словарь ответ на двадцать седьмой вопрос
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

# Submission

In [283]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [284]:
# и убедиться что ни чего не пропустил)
len(answers)

27